<a href="https://colab.research.google.com/github/TiaErikaDev/INDICIUM_Desafio_Cientista_Dados/blob/main/Modelagem_RandomForest%5BINDICIUM%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Desafio Cientista de Dados**


---



#### **Erika Mayara de Oliveira Pimentel**


---


#### Descrição do Desafio:

> Seu objetivo é identificar quais máquinas apresentam potencial de falha tendo como base dados extraídos através de sensores durante o processo de manufatura.  Para isso são fornecidos dois datasets: um dataset chamado desafio_manutencao_preditiva_treino composto por 6667 linhas, 9 colunas de informação (features) e a variável a ser prevista (“failure_type”). O segundo dataset chamado de desafio_manutencao_preditiva_teste possui 3333 linhas e 8 colunas e não possui a coluna “failure_type”. Seu objetivo é prever essa coluna a partir dos dados enviados.



---
#### Objetivo:


> Identificar quais máquinas apresentam potencial de falha.

---
#### Variáveis:
 - **UID:** identificador único que varia de 1 a 10000.
 - **product_id:** consistindo em uma letra L, M ou H para baixo (50% de todos os produtos), médio (30%) e alto (20%) como variantes de qualidade do produto e um número de série específico da variante.
 - **type:** apenas o tipo de produto L, M ou H da coluna 2.
 - **air_temperature_k:** gerada usando um processo de passeio aleatório posteriormente normalizado para um desvio padrão de 2 K em torno de 300 K.
 - **process_temperature_k:** gerada usando um processo de passeio aleatório normalizado para um desvio padrão de 1 K, adicionado à temperatura do ar mais 10 K.
 - **rotational_speed_rpm:** calculada a partir de uma potência de 2860 W, sobreposta com um ruído normalmente distribuído.
 - **torque_nm:** os valores de torque são normalmente distribuídos em torno de 40 Nm com SD = 10 Nm e sem valores negativos.
 - **tool_wear_min:** As variantes de qualidade H/M/L adicionam 5/3/2 minutos de desgaste da ferramenta à ferramenta utilizada no processo.


---


#### Entregas:

2.   Explique como você faria a previsão do tipo de falha a partir dos dados. Quais variáveis e/ou suas transformações você utilizou e por quê? Qual tipo de problema estamos resolvendo (regressão, classificação)? Qual modelo melhor se aproxima dos dados e quais seus prós e contras? Qual medida de performance do modelo foi escolhida e por quê?

# **Modelagem de Dados**


---

In [1]:
import pandas as pd
import numpy as np
import sklearn

# Ler os dados de treinamento
df_train = pd.read_csv("desafio_manutencao_preditiva_treino.csv")

# Ler os dados de teste
df_test = pd.read_csv("desafio_manutencao_preditiva_teste.csv")

# Visualizar as primeiras linhas do dataframe de treinamento
df_train.head()


,udi,product_id,type,air_temperature_k,process_temperature_k,rotational_speed_rpm,torque_nm,tool_wear_min,failure_type
0,1,M14860,M,298.1,308.6,1551,42.8,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,No Failure
2,5,L47184,L,298.2,308.7,1408,40.0,9,No Failure
3,6,M14865,M,298.1,308.6,1425,41.9,11,No Failure
4,7,L47186,L,298.1,308.6,1558,42.4,14,No Failure


In [2]:
# Visualizar as primeiras linhas do dataframe de teste
df_test.head()

,udi,product_id,type,air_temperature_k,process_temperature_k,rotational_speed_rpm,torque_nm,tool_wear_min
0,446,L47625,L,297.5,308.6,1793,26.7,70
1,7076,L54255,L,300.7,310.5,1536,47.4,192
2,1191,L48370,L,297.2,308.4,1460,42.1,41
3,2618,L49797,L,299.4,309.1,1670,35.9,68
4,5067,L52246,L,304.1,313.1,1550,30.9,9


### **1. Limpeza e Organização dos dados**




In [3]:
# Selecionar coluna 'udi' como Índice
df_train = df_train.set_index('udi')

In [4]:
# Modificar o nome da coluna índice
df_train = df_train.rename_axis('rowNumber')

In [5]:
# Selecionar coluna 'udi' como Índice
df_test = df_test.set_index('udi')

In [6]:
# Modificar o nome da coluna índice
df_test = df_test.rename_axis('rowNumber')

In [7]:
#Selecionar as colunas relevantes para o problema
df_train = df_train.drop(['product_id', 'type'], axis=1)

In [8]:
#Selecionar as colunas relevantes para o problema
df_test = df_test.drop(['product_id', 'type'], axis=1)

In [9]:
# Transformar os valores 'failure_type' para binário
df_train.loc[df_train.failure_type!='No Failure', 'failure_type']='Failure'

In [10]:
# Transformar as variáveis respostas em numéricas
df_train.loc[:,'failure_type'] = df_train.failure_type.replace('No Failure',0).replace('Failure',1)
df_train

,air_temperature_k,process_temperature_k,rotational_speed_rpm,torque_nm,tool_wear_min,failure_type
rowNumber,,,,,,
1,298.1,308.6,1551,42.8,0,0
2,298.2,308.7,1408,46.3,3,0
5,298.2,308.7,1408,40.0,9,0
6,298.1,308.6,1425,41.9,11,0
7,298.1,308.6,1558,42.4,14,0
...,...,...,...,...,...,...
9995,298.8,308.3,1634,27.9,12,0
9996,298.8,308.4,1604,29.5,14,0
9997,298.9,308.4,1632,31.8,17,0


### **2. Separação em variáveis preditivas e variável resposta**

In [11]:
X = df_train.drop('failure_type', axis=1)
y = df_train.failure_type

### **3. Separação em treino e teste**

In [12]:
from sklearn.model_selection import train_test_split

X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size=.3, random_state=61658)

### **4. Treinar o modelo**

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

params = {
      'criterion': ['gini', 'entropy'],
      'max_depth': [3,5,7,11],
}

rf = GridSearchCV(
    RandomForestClassifier(n_estimators=500, random_state=61658, n_jobs=2),
    params,
    cv=5,
    scoring='roc_auc',
)

rf.fit(X_tr, y_tr)

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(n_estimators=500, n_jobs=2,
                                              random_state=61658),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [3, 5, 7, 11]},
             scoring='roc_auc')

### **5. Validação do modelo**

In [14]:
# Verificar a métrica AUC
from sklearn.metrics import roc_auc_score

preds = rf.predict_proba(X_ts)[:,1]
roc_auc_score(y_ts, preds)

0.9833394984094104

In [15]:
# Avaliar a importância das variáveis

imps = rf.best_estimator_.feature_importances_
cols = X_tr.columns
order = np.argsort(imps)[::-1]

for col, imp in zip(cols[order], imps[order]):
  print(f'{col:30s} {imp:.2f}')

torque_nm                      0.39
rotational_speed_rpm           0.24
tool_wear_min                  0.14
air_temperature_k              0.12
process_temperature_k          0.10


### **6. Predição do modelo na tabela teste**

In [16]:
# Realiza a predição com o modelo treinado na tabela de teste
pred_test = rf.predict(df_test)


In [17]:
# Adicionar a coluna com os valores preditos na tabela de teste
df_test['predictedValues'] = pred_test


In [18]:
# Transformar as variáveis preditas em categóricas
df_test.loc[:,'predictedValues'] = df_test.predictedValues.replace(0,'No Failure').replace(1,'Failure')


In [19]:
# Selecionar as colunas de interesse para a entrega do desafio
df_entrega = df_test['predictedValues'].to_frame()
df_entrega

,predictedValues
rowNumber,
446,No Failure
7076,No Failure
1191,No Failure
2618,No Failure
5067,No Failure
...,...
5554,No Failure
6961,No Failure
6914,No Failure


In [21]:
# Salvar o dataframe em arquivo .CSV
df_entrega.to_csv('predicted.csv')

# **Conclusão**
---
> A previsão do tipo de falha a partir dos dados envolve a exploração dos dados, para entender sua distribuição, possíveis outliers e a relação entre as variáveis. O Pré-processamento dos dados com o objetivo de garantir que os dados estejam em um formato adequado para a modelagem. A Seleção de variáveis mais relevantes para prever o tipo de falha. A escolha do modelo vai depender das características dos dados e das necessidades específicas do projeto. Realizar o treinamento do modelo, avaliação do modelo com o modelo treinado e validado, é possível avaliar seu desempenho com um conjunto de dados de teste. A medida de desempenho escolhida pode ser a acurácia, precisão, recall ou outra adequada ao problema em questão. E finalmente, a previsão do tipo de falha para novos dados.

>Para prever o tipo de falha a partir dos dados, foi utilizado as seguintes variáveis:
* **air temperature [K]:** temperatura do ar em Kelvin durante o processo de manufatura.
* **process temperature [K]:** temperatura do processo em Kelvin durante o processo de manufatura.
* **rotational speed [rpm]:** velocidade de rotação da máquina em rotações por minuto.
* **torque [Nm]:** torque gerado pela máquina durante o processo de manufatura.
* **tool wear [min]:** tempo de desgaste da ferramenta durante o processo de manufatura.

> Para prever o tipo de falha a partir dos dados, estamos resolvendo um problema de **Classificação**, em que a variável de interesse é categórica, ou seja, apresenta diferentes categorias de falhas possíveis. As categorias de falha podem ser representadas por números ou por rótulos, dependendo da escolha do modelo de machine learning e da métrica de avaliação escolhida. Para o problema proposto decidi optar por um modelo de **Random Forest** por ser simples, fácil e rápido.

> Random Forest: é um modelo de conjunto (ensemble) de árvores de decisão. Random Forest é uma melhoria em relação a árvores de decisão, pois são menos propensas a overfitting e podem lidar com dados categóricos e numéricos. O Random Forest é um modelo robusto, com bom desempenho em muitos tipos de dados. Apesar de suas muitas vantagens, o modelo Random Forest também apresenta algumas desvantagens, como a interpretabilidade limitada, o Random Forest pode produzir muitas árvores de decisão, o que pode levar a um modelo grande e exigir mais recursos computacionais e de armazenamento. Pode ser sensível a dados ruidosos.

> A métrica de validação escolhida foi a métrica roc_auc_score que é uma função utilizada para calcular a área sob a curva ROC (ROC AUC) de um modelo de classificação binária. Essa métrica avalia a capacidade do modelo de distinguir entre classes positivas e negativas e é uma métrica amplamente utilizada para avaliar modelos de classificação, especialmente em problemas em que as classes estão desbalanceadas e a taxa de falsos positivos e falsos negativos não têm o mesmo peso na avaliação.   




